In [1]:
import torch
import gecco_torch
from gecco_torch.scatter import plot

In [2]:
config_root = '/groups/turaga/home/troidlj/gecco/gecco-torch/example_configs'
config = gecco_torch.load_config(f'{config_root}/neuron_unconditional.py') # load the model definition
model = config.model
state_dict = torch.load(f'{config_root}/lightning_logs/version_2/checkpoints/epoch=7-step=40000.ckpt', map_location='cuda')
model.load_state_dict(state_dict['ema_state_dict'])
model = model.eval()

In [5]:
samples = model.sample_stochastic(
    (1, 2048, 3), # one example with 2048 3-dimensional points
    context=None, # assuming an unconditional model
    with_pbar=True, # shows a tqdm progress bar for sampling
)

  0%|          | 0/64 [00:00<?, ?step/s]

In [ ]:
plot(samples.squeeze(0).cpu().numpy())

: 